mergingtariffandindustrydata.py

Jay Sayre - sayrejay (at) Gmai|,

Purpose: 
Converts tariff data (processed by wtoandcaftatariffcleaning.py)
to ISIC two digit averages

Also combines tariff data (processed by wtoandcaftatariffcleaning.py) and estimated shares of industrial activity in a given municipality (processed by compute_regional_employment.py)

INPUTS -

'Tariff Conversion/HS1996toISIC3/HS1996toISIC3.csv' - Not sure
what file made this, source seems to be 
http://wits.worldbank.org/product_concordance.html

estmunicipalindustryactivity2002.csv - Estimates municipal industrial activity at the ISIC 2-digit level for 2002 (using D.R. empresa and IPUMS data) processed by compute_regional_employment.py

estmunicipalindustryactivity2010.csv - Estimates municipal industrial activity at the ISIC 2-digit level for 2010

"cafta-dr/Output/wtoandcaftahstariff.csv" - Harmonized system 6-digit level tariff data from 1996-2015 from both WTO data and CAFTA treaty text processed by wtoandcaftatariffcleaning.py

INTERMEDIATE DATA -

OUTPUTS - 

"cafta-dr/Output/ISICtwodigitleveltariffs.csv" - tariff averages at the ISIC 2-digit level for 2002 and 2013

"cafta-dr/Output/ISICtwodigitleveltariffs.csv" - tariff averages at the ISIC 4-digit level for 2002 and 2013, updated on 2/13/16
    
"cafta-dr/Output/municipalityaveragetariff2002.csv" - municipality level tariff averages (for import competing industries, where import
competing is considered harmonized system goods that correspond to ISIC
codes based upon conversion table) for the D.R. in 2002, using estimated industrial activity in a municipality for 2002 produced by
the script compute_regional_employment.py

"cafta-dr/Output/municipalityaveragetariff2013.csv" - municipality level tariff averages (for import competing industries, where import
competing is considered harmonized system goods that correspond to ISIC
codes based upon conversion table) for the D.R. in 2013, using estimated industrial activity in a municipality for 2010 produced by
the script compute_regional_employment.py



In [1]:
import pandas as pd
import os

if os.name == 'nt':
    base_dir = "D:/Dropbox/Dropbox (Personal)/College/DR_Paper/"
else:
    base_dir = "/home/j/Dropbox/College/DR_Paper/"

## INPUTS
hs96isic3 = base_dir + 'Tariff Conversion/HS1996toISIC3/HS1996toISIC3.csv'
wtoandcaftadata = base_dir +"cafta-dr/Output/wtoandcaftahstariff.csv"
industry2002 = base_dir+'cafta-dr/Output/estmunicipalindustryactivity2002.csv'
industry2010 = base_dir+'cafta-dr/Output/estmunicipalindustryactivity2010.csv'

## INTERMEDIATE DATA

## OUTPUTS
isictariffavg = base_dir+"cafta-dr/Output/ISICtwodigitleveltariffs.csv"
isic4digtariffavg = base_dir+"cafta-dr/Output/ISICfourdigitleveltariffs.csv"
munavgtariff2002 = base_dir+"cafta-dr/Output/municipalityaveragetariff2002.csv"
munavgtariff2013 = base_dir+"cafta-dr/Output/municipalityaveragetariff2013.csv"

In [2]:
### Convert wtoandcaftadata from Harmonized System 1996 to ISIC four digit codes

## Build table to convert HS1996 to ISIC3 data
isic3 = pd.read_csv(hs96isic3, 
                    converters={'HS 1996 Product Code': lambda x: str(x),
                                'ISIC Revision 3 Product Code': lambda x: str(x)})
isic3.columns=['HS96', 'HSdesc', 'ISIC3', 'ISICdesc']
isic3 = isic3[['HS96', 'ISIC3']]

isic3conv = dict(zip(isic3['HS96'],isic3['ISIC3']))

## Convert tariff data from HS1996 to ISIC3
tariffdf = pd.read_csv(wtoandcaftadata)
#Drop number of tariff lines per HS6 good
lines = [a for a in tariffdf.columns if 'Lines' in a] 
tariffdf = tariffdf.drop(lines, 1)

#Convert HS1996 codes in tariff data into ISIC3 codes
tariffdf['ISIC'] = tariffdf['HS6'].apply(lambda x: "'"+isic3conv[x.replace("'","")])

### Group tariff data down to the ISIC two digit level
## This is a really unsophisticated way to group data
tariffdf['ISICtwodig'] = tariffdf['ISIC'].apply(lambda x: str(int(x[1:3])))
tariffdf.drop(['HS6','ISIC'],1,inplace=True)
tariffdf = tariffdf.groupby('ISICtwodig', as_index=False)[list(tariffdf.columns)].mean()
## Keep only columns that need to be merged with industrial activity data
tariffdrops = set(tariffdf.columns)-set(['ISICtwodig','2002AvgRate','duty2013'])
tariffdf = tariffdf.drop(list(tariffdrops),1)
tariffdf.rename(columns={'ISICtwodig':'isic'},inplace=True)
## We don't want ISIC code 99, serves as a catchall term in conversion
tariffdf = tariffdf[tariffdf['isic'] != '99']

In [3]:
## Checking merging issues - unfortunately, seems unfixable
print "ISIC 2 digit codes in conversion data:", 
print len(set([int(a[:2]) for a in isic3conv.values()]))
print "ISIC Codes:", set([int(a[:2]) for a in isic3conv.values()])
print "ISIC 2 digit codes in tariff data:", len(list(tariffdf['isic']))
print "ISIC 2 digit codes in industrial activity data:",
#print len([a for a in list(indusdf2002.columns) if len(a) <= 2])

ISIC 2 digit codes in conversion data: 35
ISIC Codes: set([1, 2, 5, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 40, 74, 92, 93, 99])
ISIC 2 digit codes in tariff data: 34
ISIC 2 digit codes in industrial activity data:


In [4]:
### Merge tariff data with municipality level occupation data
### Used to compute municipality level average of import competing tariff

indusdf2002  =  pd.read_csv(industry2002)
indusdf2010  =  pd.read_csv(industry2010) 

## Drop ISIC 2 digit codes that don't have corresponding tariff data
isicintariffdf = list(tariffdf['isic'])
indusdf2002drops = [a for a in indusdf2002.columns if a not in isicintariffdf]
indusdf2010drops = [a for a in indusdf2010.columns if a not in isicintariffdf]
indusdf2002drops.remove('mun')
indusdf2010drops.remove('mun')
indusdf2002 = indusdf2002.drop(list(indusdf2002drops),1)
indusdf2010 = indusdf2010.drop(list(indusdf2010drops),1)
##Compute total number of sample of workers in given municipality
codes2002 = [a for a in indusdf2002.columns if 'mun' not in a]
codes2010 = [a for a in indusdf2010.columns if 'mun' not in a]
indusdf2002['MUNTOTAL']=0
indusdf2010['MUNTOTAL']=0
for col in codes2002:
    indusdf2002['MUNTOTAL'] += indusdf2002[col]
for col in codes2010:
    indusdf2010['MUNTOTAL'] += indusdf2010[col]
## Compute share of occupation in a given municipality
for col in codes2002:
    indusdf2002[col] = indusdf2002[col]/indusdf2002['MUNTOTAL']
for col in codes2010:
    indusdf2010[col] = indusdf2010[col]/indusdf2010['MUNTOTAL']
indusdf2002 = indusdf2002.drop('MUNTOTAL',1)
indusdf2010 = indusdf2010.drop('MUNTOTAL',1)

## Pivot municipality level occupation data.
indusdf2002 = indusdf2002.set_index('mun').T.reset_index()
indusdf2010 = indusdf2010.set_index('mun').T.reset_index()
indusdf2002.rename(columns={'index':'isic'},inplace=True)
indusdf2010.rename(columns={'index':'isic'},inplace=True)

## Merge
indusdf2002 = indusdf2002.merge(tariffdf, on='isic',how='left')
indusdf2010 = indusdf2010.merge(tariffdf, on='isic',how='left')

## Compute average tariff in 2002 and 2013 for a given municipality
cols02=set(list(indusdf2002.columns))-set(['isic','2002AvgRate','duty2013'])
cols10=set(list(indusdf2010.columns))-set(['isic','2002AvgRate','duty2013'])
cols02 = sorted(list(cols02))
cols10 = sorted(list(cols10))
for col in cols02:
    indusdf2002[str(col)+'duty2002']=indusdf2002[col]*indusdf2002['2002AvgRate']
for col in cols10:    
    indusdf2010[str(col)+'duty2013']=indusdf2010[col]*indusdf2010['duty2013']
    
dropcols02=cols02+['isic','2002AvgRate','duty2013']
dropcols10=cols10+['isic','2002AvgRate','duty2013']
indusdf2002.drop(dropcols02,1,inplace=True)
indusdf2010.drop(dropcols10,1,inplace=True)
indusdf2002 = indusdf2002.sum()
indusdf2010 = indusdf2010.sum()
indusdf2002 = indusdf2002.reset_index()
indusdf2010 = indusdf2010.reset_index()
indusdf2002.columns = ['mun','tariff']
indusdf2010.columns = ['mun','tariff']
indusdf2002['mun'] = indusdf2002['mun'].apply(lambda x: x.replace('duty2002',''))
indusdf2010['mun'] = indusdf2010['mun'].apply(lambda x: x.replace('duty2013',''))

In [5]:
### Save outputs to file
tariffdf.to_csv(isictariffavg,index=False)
indusdf2002.to_csv(munavgtariff2002,index=False)
indusdf2010.to_csv(munavgtariff2013,index=False)